In [1]:
import numpy as np
from implementations import *
from proj1_helpers import *
import matplotlib.pyplot as plt
from costs import *
from hw_helpers import *
from cross_validation import *
from visualization import *
from gradients import*

In [20]:
def standardize(x):
    #print(np.nanmean(x, axis=0))
    centered_data = np.subtract(x, np.nanmean(x, axis=0))
    std_data = centered_data / np.nanstd(centered_data, axis=0)
    return std_data

In [21]:
def standardize_test(x_tr, x_te):
    mean = np.nanmean(x_tr, axis=0)
    centered_data = np.subtract(x_te, mean)
    std = np.nanstd(x_tr, axis=0)
    std_data = centered_data/std
    return std_data

In [22]:
def penalized_logistic_regression(y, tx, w, lambda_):
    loss = (np.log(1 + np.exp(tx@w))- (y-tx@w) - lambda_/2*(np.linalg.norm(w)*np.linalg.norm(w))).sum()
    gradient = tx.T@(sigmoid(tx@w)-y) - lambda_*w
    #print("gradient", gradient)
    return loss, gradient

In [23]:
def learning_by_penalized_gradient(y, tx, w, gamma, lambda_):
    loss, gradient = penalized_logistic_regression(y,tx, w, lambda_)
    #print("w", w)
    #print("gg", gradient*gamma)
    w = w-(gradient*gamma)
    return loss,w

In [19]:
def logistic_regression_penalized_gradient_descent_demo(y, x):
    # init parameters
    max_iter = 1000
    gamma = 0.01
    lambda_ = 0.7
    threshold = 1e-8
    losses = []

    # build tx
    w = np.random.normal(size=(x.shape[1],1))

    # start the logistic regression
    for it in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_penalized_gradient(y, x, w, gamma, lambda_)
        #print(loss.shape, w.shape)
        # log info
        if it % 100 == 0:
            print("Current iteration={i}, loss={l}".format(i=it, l=loss))
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    # visualization
    visualization(y, x, mean_x, std_x, w, "classification_by_logistic_regression_penalized_gradient_descent")
    print("loss={l}".format(l=calculate_loss(y, tx, w)))

In [9]:
yb, input_data, ids = load_csv_data('Data/train.csv')

In [10]:
yb_te, input_data_te, ids_te = load_csv_data('Data/test.csv')

In [11]:
#nan in the last colums are represented by 0
np.place(input_data[:,29], input_data[:,29] == 0, np.nan)
input_data = np.delete(input_data, [4,5,6,12,23,26,27,28],axis= 1)
np.place(input_data_te[:,29], input_data_te[:,29] == 0, np.nan)
input_data_te = np.delete(input_data_te, [4,5,6,12,23,26,27,28],axis= 1)
np.place(input_data, input_data < -998, np.nan)
np.place(input_data_te, input_data_te < -999, np.nan)

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in less
  """
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in less
  


In [13]:
#standarization
std_input = standardize(input_data)
std_input_te = standardize(input_data_te)
std_input = np.nan_to_num(std_input)
std_input_te = np.nan_to_num(std_input_te)

In [15]:
yb = yb.reshape(yb.shape[0], -1)

In [16]:
yb_te = yb_te.reshape(yb_te.shape[0], -1)

In [24]:
logistic_regression_penalized_gradient_descent_demo(yb, std_input)

Current iteration=0, loss=-1295493.7222941334


/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  
/Users/lucasgauchoux/Documents/MachineLearning/MLrepo/ML_HW/hw_helpers.py:31: RuntimeWarning: overflow encountered in exp
  return 1/(1 + np.exp(-t))
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in double_scalars


Current iteration=100, loss=inf
Current iteration=200, loss=inf
Current iteration=300, loss=inf
Current iteration=400, loss=inf


KeyboardInterrupt: 